<a href="https://colab.research.google.com/github/krislars/R-uncertainty/blob/master/MonteCarloModeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import numpy as np
import numpy.random as npRand
from matplotlib import pyplot as plt
import scipy.stats as stats

In [2]:
%matplotlib inline

In [281]:
#Defining some parameters:
numVals = 10000
diffABmin = 0.9
diffABmax = 1.1

In [282]:
#MultiDimensional linspace
def ndlinspace(start, end, steps):
    if (start.shape != end.shape):
        print("Arrays must be same size")
        return
    if (start.ndim == 1):
        result = np.array(
            [np.linspace(s, e, steps) for s,e in zip(start, end)]
        )
        return result
    result = np.array(
        [ndlinspace(s, e, steps) for s,e in zip(start, end)]
    )
    return result

Data file format is intrinsic stellar H-K, sigma H-K, J-H, sigma J-H ... 
Davenport, J. R., Ivezic, Z., Becker, A. C., Ruan, J. J., Hunt-Walker, N. M., Covey, K. R., & Lewis, A. R. (2014, June). The SDSS-2MASS-WISE 10-dimensional stellar colour locus [Electronic version]. MNRAS, 440(4), 3430-3438.

$$
R = \frac{A-B}{C-D}
$$

We will choose values such that the numerator is diffAB.  So, $A=(diffAB)+B$.  We also choose the value of R to be 1.6, so $C-D=(diffAB)/1.6$.  Therefore, $C = (diffAB)/1.6+D$.

In [283]:
starData = np.loadtxt("https://raw.githubusercontent.com/krislars/R-uncertainty/master/Astro%20Lab%20Star%20Data.txt")
starData = np.transpose(starData)
B, dB = starData[0], starData[1]
D, dD = starData[2], starData[3]

diffAB = np.linspace(diffABmin, diffABmax, 100)
diffAB = np.transpose(ndlinspace(diffAB, diffAB, 50))

B, dB = ndlinspace(B,B,100), ndlinspace(dB,dB,100)
D, dD = ndlinspace(D,D,100), ndlinspace(dD,dD,100)

A = diffAB + B
C = diffAB/1.6 + D

dA = dC = np.linspace(0.02, 0.02, 50)
dA = dC = ndlinspace(dA, dA, 100)

This next line is propagation of error in an arithmetic function.

In [284]:
#calculating theoretical dR
dR = 1.6 *(dB**2 + dA**2 + 1.6 *(dD**2 + dC**2))**0.5

This function takes a vector of values and a vector of associated uncertainties and returns an array of n = numVals samples selected randomly from a normal  probability distribution.

In [285]:
#choosing random data sets for each star type
def ndNormalData(mean, sigma, numVals):
    if (mean.shape != sigma.shape):
        print("Arrays must be same size")
        return
    if (mean.ndim == 1):
        data = np.array(
            [npRand.normal(m, s, numVals) for m, s in zip(mean, sigma)]
        )
        return data
    
    data = np.array(
        [ndNormalData(m, s, numVals) for m, s in zip(mean, sigma)]
    )
    return data

In [286]:
B_vals = ndNormalData(B, dB, numVals)
D_vals = ndNormalData(D, dB, numVals)
A_vals = ndNormalData(A, dA, numVals)
C_vals = ndNormalData(C, dC, numVals)

In [287]:
#calculating the "Monte Carlo" values for R and dR
R_vals = (A_vals - B_vals) / (C_vals - D_vals)
R_vals.shape

(50, 100, 10000)

Now, we can test the distributions of R.

In [288]:
R_mc = np.mean(R_vals, axis=2)
dR_mc = np.std(R_vals, axis=2)

In [289]:
#Finding the weighted average of R

w = (dR_mc)**(-2)
R_avg = sum(w*R_mc) / sum(w)
dR_avg = (sum(w))**-1
#
#t = np.linspace(0, 50, 200)
#x = np.linspace(1, 50)
#y = np.linspace(R_avg, R_avg, 200)
#
#plt.figure(figsize=(15, 5))
#line1 = plt.errorbar(x, R_mc, yerr=dR_mc, fmt='ro', label='1')
#line2 = plt.errorbar(t, y, dR_avg, fmt='b-', label='2')
#
#plt.title('Reddening Factor vs Star Type')
#plt.xlabel('Star Type')
#plt.ylabel('Mean Reddening Factor')
#plt.gca().legend(('Mean Reddening Factor', 'Weighted Average of the Means'), loc = 'upper left')
#
#plt.show() 

#Need to modify for multiple diffAB values

Ideas going forward:
* Calculate the uncertainty in the mean.
* Try a weighted mean



In [290]:
def ndHistogramCounts(Vals, numBins):
    if (Vals.ndim == 1):
        counts, Bins = np.histogram(Vals, numBins)
        return counts
    
    results = np.stack(
        ndHistogramCounts(vals, numBins) for vals in Vals
    )
    
    return results

def ndHistogramBins(Vals, numBins):
    if (Vals.ndim == 1):
        counts, Bins = np.histogram(Vals, numBins)
        return Bins
    
    results = np.stack(
        ndHistogramBins(vals, numBins) for vals in Vals
    )
    
    return results

def ndNormalCounts(mean, sigma, Bins):
    if (mean.shape != sigma.shape):
        print("Arrays must be same shape")
        return
    
    if (mean.ndim == 1):
        counts = np.diff(np.stack(
            stats.norm.cdf(bins, m, s) for bins, m, s in zip(Bins, mean, sigma)
        ))
        return counts
    
    counts = np.stack(
        ndNormalCounts(m, s, bins) for m, s, bins in zip(mean, sigma, Bins)
    )
    return counts


In [292]:
#Next I need to test "Goodness of fit"
#try writing seperate functions to get counts and bins
numBins = 20
counts = ndHistogramCounts(R_vals, numBins)
Bins = ndHistogramBins(R_vals, numBins)

#calc expected with stats cdf and np.diff
exp = ndNormalCounts(R_mc, dR_mc, Bins)

#scale expected percentages by sample size
exp = numVals * exp
exp
#Now calculate the Chi^2 values:
chi2 = np.sum(( (counts - exp)**2 / exp), axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in true_divide
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  


In [ ]:
#Plotting data sets with large Chi^2; looking for outliers
red_chi2 = chi2/numBins

dataToCheck = []
for i in range(len(chi2)):
    if red_chi2[i] > 10:
        dataToCheck.append(i)

for i in dataToCheck:
    plt.hist(R_vals[i], Bins[i], density=True)
    plt.xlabel("Reddening Factor")
    plt.ylabel("Density")
    plt.show()

dataToCheck